In [1]:
import utils
from scipy.ndimage import filters, measurements
import matplotlib.pyplot as plt
import cv2
from skimage import measure,color
from skimage.morphology import remove_small_objects, watershed
import json
from openslide import OpenSlide
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import scipy.misc
import scipy.io as sio
import numpy as np
import os

In [2]:
slide_path = '/home/bnc/tool/HistomicsML/yourproject/hover_net/slide/'
json_path = '/home/bnc/tool/HistomicsML/yourproject/hover_net/json/'
recentage_path = '/home/bnc/tool/HistomicsML/yourproject/hover_net/recentage/'

In [3]:
list_slide =   [os.path.join(slide_path,slide) for slide in os.listdir(slide_path)] 
list_json = [os.path.join(json_path,json) for json in os.listdir(json_path)]
print(list_slide)
print(list_json)

['/home/bnc/tool/HistomicsML/yourproject/hover_net/slide/2018-49984003.ndpi', '/home/bnc/tool/HistomicsML/yourproject/hover_net/slide/2018-50049004.ndpi']
['/home/bnc/tool/HistomicsML/yourproject/hover_net/json/2018-50049004_new_cls.json', '/home/bnc/tool/HistomicsML/yourproject/hover_net/json/2018-49984003_new_cls.json']


In [4]:
info1 = utils.openjson(list_json[0])
info2 = utils.openjson(list_json[1])
print(len(info2))

3660


In [5]:
def get_cor(info):
    ans = []
    for reigon in info:
        points = reigon['points']
        X = []
        Y = []
        i = 0
        while(i<len(points)-1):
            X.append(points[i])
            Y.append(points[i+1])
            i = i + 2
        nuclei = {}
        nuclei['x_zuobiao'] = X
        nuclei['y_zuobiao'] = Y
        ans.append(nuclei)
    return ans


In [6]:
def get_size(ans):
    X = []
    Y = []
    for nuclei in ans:
        X = X + nuclei['x_zuobiao']
        Y = Y + nuclei['y_zuobiao']
    res_x_min = min(X)
    res_x_max = max(X)
    res_y_min = min(Y)
    res_y_max = max(Y)
    return res_x_min,res_x_max,res_y_min,res_y_max
    

In [7]:
def tiaozheng(ans):
    x_min,x_max,y_min,y_max = get_size(ans)
    for nuclei in ans:
        nuclei['x_zuobiao'] = [data - x_min for data in nuclei['x_zuobiao']]
        nuclei['y_zuobiao'] = [data - y_min for data in nuclei['y_zuobiao']]

In [8]:
def crop_box(rencentage_path):
    list_recentage = [os.path.join(rencentage_path,json) for json in os.listdir(recentage_path)]
    print(list_recentage)
    for recentage in list_recentage:
        file_rec = os.path.basename(recentage).split('_')[0]
        for slide in list_slide:
            file_slide = os.path.basename(slide).split('.')[0]
            if file_rec == file_slide:
                info = utils.openjson(recentage)
                ans = get_cor(info)
                x_min,x_max,y_min,y_max = get_size(ans)
                w = int(x_max-x_min)
                h = int(y_max-y_min)
                x_min = int(x_min)
                y_min = int(y_min)
                slide = OpenSlide(slide)
                tile = slide.read_region((x_min,y_min),0, (w,h))
                tile = cv2.cvtColor(np.asarray(tile),cv2.COLOR_RGB2BGR)
                cv2.imwrite('./recentage/{}.png'.format(file_slide),tile)
                print(get_size(ans))
                
    

In [12]:
def filter_nuclei(recentage_path,json_path):
#     list_recentage = [os.path.join(rencentage_path,json) for json in os.listdir(recentage_path)]
    list_json = [os.path.join(json_path,json) for json in os.listdir(json_path)]
    for recentage in list_recentage:
        file_rec = os.path.basename(recentage).split('_')[0]
        for json in list_json:
            file_json = os.path.basename(json).split('_')[0]
            if file_rec == file_json:
                print(file_rec)
    

In [13]:
def main():
    #crop_box(recentage_path)
    filter_nuclei(recentage_path,json_path)
#     print(list_slide[0])
#     print(list_json[1])
#     slide = OpenSlide(list_slide[1])
#     ans = get_cor(info1)
#     print(ans[0])
#     x_min,x_max,y_min,y_max = get_size(ans)
#     w = int(x_max-x_min)
#     h = int(y_max-y_min)
#     x_min = int(x_min)
#     y_min = int(y_min)
#     print(x_min)
#     print(y_min)
#     tile = slide.read_region((x_min,y_min),0, (w,h))
#     tile = cv2.cvtColor(np.asarray(tile),cv2.COLOR_RGB2BGR)
#     cv2.imwrite('./crop/tile0.png',tile)
#     plt.imshow(tile)
#     plt.show()
#     print(w,h)
#     tiaozheng(ans)
#     x_min,x_max,y_min,y_max = get_size(ans)
#     w = x_max-x_min
#     h = y_max-y_min
#     print(ans[0])
#     print(w,h)
#     for nuclei in ans:
        
#         x_cor = nuclei['x_zuobiao']
#         y_cor = nuclei['y_zuobiao']
#         print(len(x_cor))
#         print(len(y_cor))
#         res = []
#         for i in range(0,len(x_cor)):
#             temp = []
#             temp.append(x_cor[i])
#             temp.append(y_cor[i])
#             res.append(temp)
#         res = np.array(res,dtype = np.int32)
#         print(res.shape)
#         cv2.polylines(tile,np.int32([res]),True,(0,255,0),thickness=3)
#     cv2.imwrite('./crop/tile0_result.png',tile)
        

In [14]:
if __name__ =='__main__':
    main()

NameError: name 'list_recentage' is not defined